https://huggingface.co/docs/transformers/en/training

In [ ]:
!pip install -qq adapters datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 66.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("hrithikpiyush/scierc")
dataset = dataset.rename_column(original_column_name="label", new_column_name="labels")

train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3219 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/455 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/974 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/cs_roberta_base")

def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

tokenized_train_data = train_data.map(tokenize_function, batched=True, batch_size=len(train_data))
tokenized_val_data = val_data.map(tokenize_function, batched=True, batch_size=len(val_data))
tokenized_test_data = test_data.map(tokenize_function, batched=True, batch_size=len(test_data))

tokenizer_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Map:   0%|          | 0/3219 [00:00<?, ? examples/s]

Map:   0%|          | 0/455 [00:00<?, ? examples/s]

Map:   0%|          | 0/974 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = len(set(train_data['labels']))
model = AutoModelForSequenceClassification.from_pretrained("allenai/cs_roberta_base", num_labels=num_labels)

pytorch_model.bin:   0%|          | 0.00/656M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allenai/cs_roberta_base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
import evaluate

# https://huggingface.co/spaces/evaluate-metric/f1/blob/main/f1.py
# Example 5-A multi-label example
metric = evaluate.load("f1", "multilabel")

In [ ]:
# https://medium.com/@rakeshrajpurohit/customized-evaluation-metrics-with-hugging-face-trainer-3ff00d936f99

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision_macro = precision_score(labels, preds, average='macro')
    precision_weighted = precision_score(labels, preds, average='weighted')
    recall_macro = recall_score(labels, preds, average='macro')
    recall_weighted = recall_score(labels, preds, average='weighted')
    f1_macro = f1_score(labels, preds, average='macro')
    f1_weighted = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'precision_weighted': precision_weighted,
        'recall_macro': recall_macro,
        'recall_weighted': recall_weighted,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    learning_rate=1e-4,  # <- follow paper for smaller dataset
    num_train_epochs=5,  # <- paper used 100 epoch for TAPT instead
    per_device_train_batch_size=128,  # <- follow paper for smaller dataset
    per_device_eval_batch_size=128,  # <- follow paper for smaller dataset
    auto_find_batch_size=True,  # automates lowering the batch size to resolve out-of-memory errors
    adam_beta1=0.9,  # <- follow paper
    adam_beta2=0.98,  # <- follow paper
    adam_epsilon=1e-6,  # <- follow paper
    lr_scheduler_type='constant',  # <- follow paper for smaller dataset
    weight_decay=0.01,  # <- follow paper
    warmup_ratio=0.06,  # <- follow paper
    logging_steps=100,  # <- follow paper
    output_dir="./training_output_3",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy="epoch"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Precision Weighted,Recall Macro,Recall Weighted,F1 Macro,F1 Weighted
1,0.647100,0.489442,0.854945,0.862139,0.873318,0.760964,0.854945,0.770993,0.842609
2,0.436700,0.452228,0.870330,0.820983,0.882114,0.855642,0.870330,0.830431,0.871612
3,0.357100,0.570583,0.874725,0.855217,0.878916,0.834706,0.874725,0.839978,0.873440
4,0.236400,0.611176,0.876923,0.844956,0.880346,0.857488,0.876923,0.847921,0.876848
5,0.267900,0.563876,0.890110,0.864576,0.896478,0.849524,0.890110,0.852826,0.891172


TrainOutput(global_step=1010, training_loss=0.43558878473716206, metrics={'train_runtime': 1554.0728, 'train_samples_per_second': 10.357, 'train_steps_per_second': 0.65, 'total_flos': 4234962547584000.0, 'train_loss': 0.43558878473716206, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.5638757944107056,
 'eval_accuracy': 0.8901098901098901,
 'eval_precision_macro': 0.8645760933435953,
 'eval_precision_weighted': 0.8964778841869143,
 'eval_recall_macro': 0.8495238243549884,
 'eval_recall_weighted': 0.8901098901098901,
 'eval_f1_macro': 0.85282631964917,
 'eval_f1_weighted': 0.8911724593841039,
 'eval_runtime': 13.9757,
 'eval_samples_per_second': 32.557,
 'eval_steps_per_second': 0.286,
 'epoch': 5.0}

In [ ]:
trainer.predict(tokenized_test_data)

PredictionOutput(predictions=array([[ 1.4149219 , -1.5862333 ,  4.7209034 , ..., -1.7715043 ,
        -1.6540197 , -2.8952157 ],
       [ 0.13466221, -1.0077913 ,  5.45806   , ..., -0.22349791,
        -1.578513  , -2.207181  ],
       [ 2.440849  , -2.527328  ,  4.5311832 , ...,  0.23348084,
        -1.776113  , -2.0889819 ],
       ...,
       [-0.23615365,  1.3141543 , -2.1653702 , ..., -0.45750216,
         4.815073  , -1.3117654 ],
       [ 0.9454919 , -1.9144635 , -0.9181931 , ..., -1.1835483 ,
        -0.5833594 , -2.5584507 ],
       [-1.4997209 ,  7.5071754 , -1.1536793 , ..., -2.7158177 ,
         0.34920067, -0.969601  ]], dtype=float32), label_ids=array([2, 2, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 2, 5, 2, 0, 0, 0, 0, 2, 0, 0,
       0, 5, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 2,
       0, 0, 4, 0, 0, 4, 4, 1, 0, 0, 0, 6, 2, 0, 0, 0, 0, 0, 5, 5, 5, 0,
       6, 5, 6, 0, 0, 3, 0, 0, 4, 0, 0, 0, 0, 5, 5, 0, 0, 0, 5, 5, 0, 0,
       5, 0, 6, 6, 6, 0, 0, 0, 0, 0, 4, 